In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
%matplotlib inline

In [2]:
df = sns.load_dataset("diamonds")
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [3]:
label_name = "cut"

In [4]:
X_raw = df.drop(label_name, axis=1)
y = df[label_name]

X_raw.shape, y.shape

((53940, 9), (53940,))

In [5]:
X = pd.get_dummies(X_raw)
X

,carat,depth,table,price,x,y,z,color_D,color_E,color_F,...,color_I,color_J,clarity_IF,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,0,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50,1,0,0,...,0,0,0,0,0,0,0,1,0,0
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61,1,0,0,...,0,0,0,0,0,0,0,1,0,0
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56,1,0,0,...,0,0,0,0,0,0,0,1,0,0
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X.values, y, test_size=0.33, random_state=42)

In [15]:
y_train.nunique()

5

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(X_train.shape[1], 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, y_train.nunique()),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [73]:
model = NeuralNetwork()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=22, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=5, bias=True)
  )
)


In [74]:
logits = model(torch.FloatTensor(X_train))
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1, 1, 1,  ..., 1, 1, 1])


In [79]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y_train_ohe = ohe.fit_transform(y_train.to_frame())
y_one_hot = y_train_ohe.toarray()
y_one_hot

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]])

In [76]:
y_train_ohe.shape

(36139, 5)

In [82]:
nb_epochs = 100
for epoch in range(nb_epochs+1):
    
    hypothesis = model(torch.FloatTensor(X_train))
#     cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    cost = F.cross_entropy(hypothesis, torch.FloatTensor(y_one_hot))
    optimizer.zero_grad()
    # gradient
    cost.backward()
    # Weight and bias update
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/100 Cost: 38.246487
Epoch   10/100 Cost: 49.461178
Epoch   20/100 Cost: 28.509354
Epoch   30/100 Cost: 24.582552
Epoch   40/100 Cost: 26.560949
Epoch   50/100 Cost: 37.170212
Epoch   60/100 Cost: 18.991268
Epoch   70/100 Cost: 19.739822
Epoch   80/100 Cost: 40.607445
Epoch   90/100 Cost: 11.472807
Epoch  100/100 Cost: 41.030624
